In [147]:
from sympy import symbols, pprint
from sympy import diff
from sympy.solvers import solve
import numpy as np
from scipy import optimize
import string
import random

# global espilon budget
overall_budget_limit = 5

# data subject
class Entity():
    def __init__(self, name="", id=None):
        self.name = name
        self.id = id
        
class AdversarialAccountant():
    
    def __init__(self):
        ""

scalar_name2obj = {}
        
class Scalar():
    
    def __init__(self, value, min_val=None, max_val=None, poly=None, ent=None, name=None):
        
        if name is None:
            lower_upper_alphabet = string.ascii_letters
            name = ''.join([random.choice(lower_upper_alphabet) for i in range(5)])
        
        self.name = ""
        self.value = value
        self.min_val = min_val
        self.max_val = max_val
        
        if poly is not None:
            self.poly = poly
        elif ent is not None:
            self.poly = symbols(self.name + "" + ent.name)
            scalar_name2obj[self.name + "" + ent.name] = self            
        else:
            raise Exception("Poly or ent must be not None")

    
    def __mul__(self, other):
        
        result_poly = self.poly * other.poly
        result_value = self.value * other.value
        
        result = Scalar(value=result_value, poly=result_poly)
        
        return result
    
    def __add__(self, other):
        result_poly = self.poly + other.poly
        result_value = self.value + other.value
        
        result = Scalar(value=result_value, poly=result_poly)
        
        return result
    
    def __sub__(self, other):
        result_poly = self.poly - other.poly
        result_value = self.value - other.value
        
        result = Scalar(value=result_value, poly=result_poly)
        
        return result
    
    def __str__(self):
        return str(self.poly) + "=" + str(self.value)
    
    def __repr__(self):
        return str(self)

#     def ent_sens(self, ent_name="Bob"):
        
    
    @property
    def sens(self):
        if self.min_val is not None and self.max_val is not None:
            return self.max_val - self.min_val

    def deriv(self, name):
        obj = scalar_name2obj[name]
        return -diff(self.poly, obj.poly)
        

    def create_run_specific_args(self):

        free_symbols_list = list(self.poly.free_symbols)
        index2symbol = list(map(lambda x:str(x), free_symbols_list))

        symbol2index = {}
        for i,sym in enumerate(index2symbol):
            symbol2index[sym] = i

        def _run_specific_args(tuple_of_args, *params):
            kwargs = {}
            for sym,i in symbol2index.items():
                kwargs[sym] = tuple_of_args[i]

            return run_specific(**kwargs)

        return _run_specific_args, index2symbol, symbol2index
    
    def eps(self,
            entity_name = 'b',
            sigma = 0.1,
            alpha = 1,
            granularity = 0.5):
        
        z = self.deriv(entity_name)

        sy_names = z.free_symbols
        sym = list()
        for sy_name in sy_names:
            sym.append(scalar_name2obj[str(sy_name)])

        run_specific_args, index2symbol, symbol2index = self.create_run_specific_args()

        rranges = list()
        for i,sym in enumerate(index2symbol):
            obj = scalar_name2obj[sym]
            rranges.append(slice(obj.min_val, obj.max_val, granularity))

        resbrute = optimize.brute(run_specific_args,rranges, full_output=False, finish=None, disp=True)
        L = resbrute[symbol2index[entity_name]]

        input_obj = scalar_name2obj[entity_name]

        e = (alpha * (L ** 2) * (input_obj.value**2)) / (2 * (sigma**2))

        return e
#     def publish(self, epsilon=0.5, mechansim=None):
#         # returns the value with noise added and updates
#         # the list of entity epsilons
#         return self.

def run(func, **kwargs):
    return func.subs(kwargs)

def run_specific(**kwargs):
    """pass in kwargs to run in fixed polynomial because this is what
    optimize.brute expects"""
    return run(z, **kwargs)

bob = Scalar(value=1, min_val=-2, max_val=2, ent=Entity(name="b"))
alice = Scalar(value=1, min_val=-1, max_val=1, ent=Entity(name="a"))
charlie = Scalar(value=2, min_val=-2, max_val=2, ent=Entity(name="c"))
dan = Scalar(value=-1, min_val=-2, max_val=2, ent=Entity(name="d"))

out = bob * bob * bob + charlie + dan + bob + alice

out.eps(entity_name="a", sigma=0.1, alpha=1, granularity=0.5)

import numpy as np

tensor = np.array([bob, alice, charlie, dan])

out = (tensor * tensor).sum()

In [148]:
out

a**2 + b**2 + c**2 + d**2=7

In [80]:
run_specific(b=30, c=30, d=30, a=28)

148

In [81]:
run_specific_args((28, 30, 30, 30))

12

In [88]:
resbrute

array([-2., -2., -2., -2.])

In [8]:
params = (2, 3, 7, 8, 9, 10, 44, -1, 2, 26, 1, -2, 0.5)
def f(z, *params):
    x, y = z
    a, b, c, d, e, f, g, h, i, j, k, l, scale = params
    return (a * x**2 + b * x * y + c * y**2 + d*x + e*y + f)

rranges = (slice(-4, 4, 0.25), slice(-4, 4, 0.25))

resbrute = optimize.brute(f, rranges, args=params, full_output=True,
                          finish=optimize.fmin)
min_input = resbrute[0]

In [289]:
bob = Scalar(value=30, min_val=-1, max_val=110, ent=Entity(name="b"))
alice = Scalar(value=28, min_val=-2, max_val=100, ent=Entity(name="a"))
charlie = Scalar(value=30, min_val=-3, max_val=120, ent=Entity(name="c"))
dan = Scalar(value=30, min_val=-3, max_val=120, ent=Entity(name="d"))

In [ ]:
# ahead of time - Bob, Alice, Charlie, and Dan have all specified that they want to remain under
# certain epsilon budgets (B parameter in Feldman et al.)

bob = duet.store['bob_age']
alice = duet.store['alice_age']
charlie = duet.store['charlie_age']
dan = duet.store['dan_age']

In [288]:
sum_age = bob * alice * charlie * dan + bob*bob + dan*dan + bob*dan + charlie * bob * 0.0000001

In [ ]:
sum_age.get(noise=0.1)

In [275]:
grad

a**3 + c*d

In [280]:
grad.free_symbols

{a, c, d}

39

In [271]:
from sympy import Interval
from sympy import solveset
from scipy import optimize

In [286]:
def run(func, **kwargs):
    return func.subs(kwargs)

from scipy import optimize

root = optimize.brentq(f, -2, 0)

In [268]:
# sensitivity of bob (also variable x)

In [270]:
grad

a**3 + c*d

In [182]:
from sympy import Symbol

In [183]:
x = Symbol('x')
solve([x >= 0.5, x <= 3, x**2 - 1], x)

Eq(x, 1.0)

In [184]:
solve([alice.poly >= 0, alice.poly <=120, grad], alice.poly)

Eq(_alice, 0)

In [77]:
input_vars = list(map(lambda x:scalar_name2obj[str(x)], z.poly.args))
    

In [50]:
scalar_name2obj['JvtNO_bob']

JvtNO_bob=30

In [38]:
z

nSnfC_bob + tdSxn_alice=58

In [16]:
class PrivateScalar:
    
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        return PrivateScalar(self.value - other.value)
    
    def __str__(self):
        return self.value
    
    def __repr__(self):
        return str(self.value)

In [17]:
x = PrivateScalar(4)
y = PrivateScalar(3)

z = x + y
z.value

1

In [18]:
xv = np.array([x,x,x,x])
yv = np.array([y,y,y,y])